In [1]:
import numpy as np
from scipy import ndimage as ndi
from skimage.io import imread
from skimage.measure import find_contours
from skimage.morphology import square, binary_dilation
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from matplotlib import pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed, cpu_count
from typing import Tuple
import tifffile as tf

In [2]:
def binary_dilation_with_fixed_overlap(
    mask: np.ndarray, dilation_radius: int
) -> Tuple[np.ndarray, np.ndarray]:
    print("# EDT")
    distance = ndi.distance_transform_edt(mask)
    print("# MAXIMA")
    coords = peak_local_max(distance,
                            threshold_abs=20,
                            min_distance=15,
                            footprint=square(30))
    peaks_mask = np.zeros(distance.shape, dtype=bool)
    peaks_mask[tuple(coords.T)] = True
    peaks_markers, _ = ndi.label(peaks_mask)
    print("# WATERSHED")
    labels = watershed(-distance, peaks_markers, mask=mask)
    print("# SPLIT MASKS")
    single_masks = np.array([i == labels for i in range(0, labels.max()+1)])
    single_masks[0, :, :] = np.zeros(single_masks.shape[1:])
    print("# DILATE")
    single_dilated_masks = np.array(Parallel(n_jobs=min(20, cpu_count()), verbose=10)(
        delayed(binary_dilation)(single_masks[i, :, :], square(30))
        for i in range(single_masks.shape[0])))
    dilation_sum = single_dilated_masks.sum(0)
    print("# CERTAIN VALUES")
    certain_pixels = np.logical_and(dilation_sum == 1, np.logical_not(labels != 0))
    certain_pixel_values = np.array([single_dilated_masks[i, :, :]*i
                                     for i in tqdm(range(single_dilated_masks.shape[0]))])
    for i in tqdm(range(certain_pixel_values.shape[0])):
        certain_pixel_values[i, np.logical_not(certain_pixels)] = 0
    print("# UNCERTAIN VALUES")
    uncertain_pixels = np.logical_and(dilation_sum > 1, np.logical_not(labels != 0))
    uncertain_coords = np.array(np.where(uncertain_pixels))
    print("# EDT")
    single_dilated_edt = np.array(Parallel(n_jobs=min(20, cpu_count()), verbose=10)(
        delayed(ndi.distance_transform_edt)(single_dilated_masks[i, :, :])
        for i in range(single_dilated_masks.shape[0])))
    print("# OUTPUT")
    dilated_labels = labels.copy()
    for i in tqdm(range(uncertain_coords.shape[1])):
        icoords = uncertain_coords.transpose()[i, :]
        overlapping_labels = np.where(single_dilated_masks[:, icoords[0], icoords[1]])[0]
        selected_label = overlapping_labels[np.argmax(
            single_dilated_edt[overlapping_labels, icoords[0], icoords[1]])]
        dilated_labels[icoords[0], icoords[1]] = selected_label
    dilated_labels += certain_pixel_values.sum(0)
    return labels, dilated_labels

In [3]:
def plot_dilation(labels: np.ndarray, dilated_labels: np.ndarray, image: np.ndarray) -> None:
    fig, ax = plt.subplots(figsize=(20, 20))
    ax.imshow(image.sum(0)*-1, cmap="binary", interpolation="none")
    for i in tqdm(range(1, labels.max()+1)):
        for contour in find_contours(labels == i, .5):
            ax.plot(contour[:, 1], contour[:, 0], linewidth=1)
        for contour in find_contours(dilated_labels == i, .5):
            ax.plot(contour[:, 1], contour[:, 0], linewidth=1, linestyle='--')
    plt.close(fig)
    return fig, ax

In [4]:
def run_series(series_id: int) -> None:
    mask = imread(f"../../data/60x_mask/original/a647_{series_id:03d}.tif").reshape((2048, 2048))
    image = imread(f"/mnt/data/Imaging/202105-Deconwolf/data_210726/60x_dw/a647_{series_id:03d}.tif")
    labels, dilated_labels = binary_dilation_with_fixed_overlap(mask, 30)
    return labels, dilated_labels, image

In [5]:
for i in range(1, 8):
    print(f">>>>> SERIES {i} <<<<<")
    labels, dilated_labels, image = run_series(i)
    fig, ax = plot_dilation(labels, dilated_labels, image)
    tf.imwrite(f"../../data/60x_mask/labels/a647_{i:03d}.labels.tiff",
               labels, software="GG20210811")
    tf.imwrite(f"../../data/60x_mask/dilated_labels_watershed/a647_{i:03d}.dilated_labels.tiff",
               dilated_labels, software="GG20210811")
    fig.savefig(f"../../data/60x_mask/dilated_labels_watershed_contours/a647_{i:03d}.contours.png")

>>>>> SERIES 1 <<<<<
# EDT
# MAXIMA
# WATERSHED
# SPLIT MASKS
# DILATE


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    5.1s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    5.4s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    9.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    9.7s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:   13.7s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:   14.2s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   18.1s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   22.2s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   26.1s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   30.1s
[Parallel(n_jobs=20)]: Done 145 out of 167 | elapsed:   34.4s remaining:    5.2s
[Parallel(n_jobs=20)]: Done 162 out of 167 | elapsed:   37.9s remaining:    1.2s
[Parallel(n_jobs=20)]: Done 167 out of 167 | elapsed:   38.3s finished
  5%|▍         | 8/167 [00:00<00:02, 77.67it/s]

# CERTAIN VALUES


100%|██████████| 167/167 [00:08<00:00, 19.79it/s]
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


# UNCERTAIN VALUES
# EDT


...

100%|██████████| 167/167 [00:06<00:00, 27.81it/s]
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


# UNCERTAIN VALUES
# EDT


[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:    3.7s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:    4.5s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:    5.5s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:    7.5s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:    8.6s
[Parallel(n_jobs=20)]: Done 145 out of 167 | elapsed:   10.1s remaining:    1.5s
[Parallel(n_jobs=20)]: Done 162 out of 167 | elapsed:   11.2s remaining:    0.3s
[Parallel(n_jobs=20)]: Done 167 out of 167 | elapsed:   11.5s finished
 10%|█         | 9812/94449 [00:00<00:00, 98118.69it/s]

# OUTPUT


100%|██████████| 166/166 [00:19<00:00,  8.48it/s]
